In [1]:
import numpy as np
import pandas as pd

import decouple
import sqlalchemy

In [2]:
#Using enviroment variables for security
user = decouple.config("db_user_mysql")
host = decouple.config('db_host_mysql')
password = decouple.config('db_password_mysql')
database = decouple.config("db_database_mysql")
connection = sqlalchemy.create_engine("mysql+mysqldb://"+user+":"+password+"@"+host+"/"+database)
# sudo apt-get install libmysqlclient-dev

In [3]:
df = pd.read_sql_table('text01', con = connection)
df.head(5).T

,0,1,2,3,4
counter,1,2,3,4,5
paciente,353,341,237,0,245
dataanamnese,2016-12-05 02:45:48,2020-07-15 18:06:50,2017-10-28 20:56:46,2021-03-06 11:38:12,2020-10-04 22:11:17
palavraschave,Voluptate recusandae porro velit harum qui.,None,None,None,None
textoanamnese,<h2>Revisão de Sistemas</h2>\nEos quidem accus...,<h2>Análise de parâmetros clínicos</h2>\nIncid...,<h2>História Patológica Pregressa (HPP)</h2>\n...,<h2>Hábitos de vida</h2>\nLaborum perspiciatis...,<h2>Alergias</h2>\nFugit id repudiandae nemo. ...
diagnostico1,Quisquam tempore ipsa odit.,None,Fugit architecto officia in.,Labore eum sit omnis quos libero.,Deserunt voluptate aliquid aut.
diagnostico2,None,None,None,Dolore corrupti consequatur.,None
timestamp,2021-07-01 18:06:12,2021-07-01 18:06:12,2021-07-01 18:06:12,2021-07-01 18:06:12,2021-07-01 18:06:12
cloud,1,0,1,1,0
update001,2021-02-24 21:08:53,2020-12-29 11:46:53,2021-04-17 12:53:00,2020-12-21 10:51:20,2020-11-01 14:04:58


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   counter        440 non-null    int64         
 1   paciente       440 non-null    int64         
 2   dataanamnese   440 non-null    datetime64[ns]
 3   palavraschave  307 non-null    object        
 4   textoanamnese  440 non-null    object        
 5   diagnostico1   398 non-null    object        
 6   diagnostico2   234 non-null    object        
 7   timestamp      440 non-null    datetime64[ns]
 8   cloud          440 non-null    int64         
 9   update001      440 non-null    datetime64[ns]
 10  ch3            440 non-null    int64         
 11  sysuser        436 non-null    float64       
dtypes: datetime64[ns](3), float64(1), int64(4), object(4)
memory usage: 41.4+ KB


In [5]:
# Colocando o nome das colunas do arquivo de acordo a documentação da iClinic
df = df.rename(
    columns = {
        "paciente":"patient_id",
        "sysuser": "physician_id",
        "dataanamnese": "date",
        "timestamp":"start_time", 
        "update001":"end_time", 
        "textoanamnese":"eventblock_text",
    }
)

In [6]:
df['patient_name'] = np.nan #tratar depois
df['tab_name'] = np.nan 
df['eventblock_name'] = np.nan 

In [7]:
# Tratamento 'patient_name'
df_patient_names = pd.read_csv('desafio-base2-output/patient.csv', usecols = ["name"])
df_patient_names.head(7).T

,0,1,2,3,4,5,6
name,Nicolas Freitas Teste,Gustavo Silveira Teste,Otávio Barros Teste,Davi Lucas Fogaça Teste,Bruna Santos Teste,Daniel Campos Teste,João Pedro Lima Teste


In [16]:
teste = df['patient_id'].astype(str).str.contains(r'^427')
print(teste.value_counts())

False    439
True       1
Name: patient_id, dtype: int64


In [13]:
teste = df['patient_id'].values
print(teste.max())

10000000


In [8]:
df['patient_id'] = df['patient_id'].replace({np.nan: 0})
df['patient_id'] = [x if x <= df_patient_names.size else 0 for x in df['patient_id'] ]
index = df['patient_id'].values
df['patient_name'] = [ df_patient_names['name'][i] if i != 0 else np.nan for i in index ]

['Bárbara Ferreira Teste', 'Maria Luiza Aragão Teste', 'Amanda da Paz Teste', nan, 'Maria Eduarda Caldeira Teste', 'Luiz Henrique Ribeiro Teste', 'Gabrielly Cardoso Teste', 'Joaquim Teixeira Teste', 'Isabella Ferreira Teste', 'Guilherme Cavalcanti Teste', 'João Guilherme Cardoso Teste', 'Maitê Correia Teste', 'André Mendes Teste', 'Ana Luiza da Rosa Teste', 'Stella Dias Teste', 'Lorena Carvalho Teste', 'Carolina Carvalho Teste', 'Gabriela da Luz Teste', 'Fernanda Dias Teste', 'Pedro Dias Teste', 'Otávio Barros Teste', 'Maria Clara Azevedo Teste', 'Nicolas Cardoso Teste', 'Maria Luiza Aragão Teste', 'Gabriela Costa Teste', 'Maria Cecília Campos Teste', 'Emanuel Monteiro Teste', 'Rafaela Dias Teste', 'Maria Fernanda Cunha Teste', 'Ana Carolina da Mata Teste', 'Brenda Campos Teste', 'Maysa Moraes Teste', 'Enrico Costa Teste', 'Maysa Moraes Teste', 'Maria Vitória Lima Teste', 'Renan Novaes Teste', 'João Felipe Alves Teste', 'Luiz Fernando Farias Teste', 'Valentina Cardoso Teste', 'Maria Cl

In [8]:
# Tratamento 'gender'
# Não tem valores nulos
df['gender'] = ['m' if x == 'M' else 'f' if x == 'F' else np.nan for x in df['gender']]
# print(df['gender'].value_counts())

In [9]:
# Tratamento 'mobile_phone'
# Não tem valores nulos
rows_t = df['mobile_phone'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int).astype('str')
# print(rows_t)
# print(rows_t.str.len().value_counts())
# como só tenho numeros de telefone a partir de 10 digitos, então o tipo de telefone mais adequado é o 'home_phone' por acreditar que é o padrão dos pacientes
df['home_phone'] = '(' + rows_t.str[-10:-8] + ')' + rows_t.str[-8:-4] + '-' + rows_t.str[-4:]
df['mobile_phone'] = np.nan

In [10]:
# Tratamento 'zip_code'
# Não tem valores nulos
rows_with_dashes = df['zip_code'].str.contains('-')
# print(rows_with_dashes.value_counts())
# tem valores contendo '-'
df['zip_code'] = [df['zip_code'][i] if x == True else df['zip_code'][i][:5]+'-'+df['zip_code'][i][5:] if x == False else np.nan for i, x in enumerate(rows_with_dashes)]

In [11]:
# Tratamento 'address'
# Não tem valores nulos
# Separando as informações e distribuindo de acordo ao seu tipo
# print(df['address'])
row_with_adress = df['address'].str.split(',')
# print(row_with_adress.str.len().value_counts())
# existem linhas contrando 1 e dois valores apenas
df['number'] = [np.nan if len(x) == 1 else x[1].strip() for x in row_with_adress]
df['address'] = [x[0].strip() for x in row_with_adress]

In [12]:
# Tratamento 'ethnicity'
# Não tem valores nulos
# print(df['ethnicity'].value_counts())
df['ethnicity'] = df['ethnicity'].replace({'Branco': 'wh', 'Amarelo': 'ye', 'Pardo': 'br','Preto': 'bl'})

In [13]:
# Tratamento 'marital_status'
# Não tem valores nulos
# print(df['marital_status'].value_counts())
df['marital_status'] = df['marital_status'].replace({'S': 'si', 'C': 'ma', 'O': 'st', 'V': 'wi','D': 'se'})
# acima foi considerada a sigla "O" como união estável

In [14]:
# Tratamento 'education'
# Não tem valores nulos
# print(df['education'].value_counts())
df['education'] = df['education'].replace({'Ensino Médio': 's', 'Superior': 'h', 'Pós-graduação': 'p', 'Especialização': 'p'})
# acima foi considerada a sigla "Pós-graduação" e "Especialização" como 'p', já que não tem nada para específicar se a pós gradução é lato sensu ou se não foi informado ter sido mestrado

In [15]:
# Tratamento 'indication'
# Todos os valores são nulos
df['indication']= 'ot'

In [16]:
# Tratamento 'active'
# Não tem valores nulos
# print(df['active'].value_counts())
df['active'] = df['active'].replace({0: 1, 1: 0})
# df['active'] = df['active'].astype(np.dtype(bool))

In [17]:
# Tratamento 'picture_filename'
df['picture_filename'] = df['picture_filename'].replace({'NONE': np.nan})
df['picture_filename'] = df['picture_filename'].fillna(np.nan)
# print(df['picture_filename'])

In [18]:
# abaixo estamos removendo os valores que não representam um endereço na pasta 'extra/'
rows_begining_with_extra = df['picture_filename'].str.contains(r'^extra')
df['picture_filename'] = [df['picture_filename'][i][6:] if x == True else 'None' for i, x in enumerate(rows_begining_with_extra)]

In [19]:
for source in df['picture_filename'].values:

    source_path = "desafio-base2/extra/"+source
    # Destination path
    destination_path = "desafio-base2-output/picture/"+source

    # Create the directory 'pictures/' if it doesn't exist

    try:
        os.path.exists('desafio-base2-output/picture/')
        os.makedirs('desafio-base2-output/picture/')
        print("Directory created.")

    # If there is any permission issue
    except:
        print("Directory already exist.")

    # Copy the content of
    # source to destination

    try:
        shutil.copy(source_path, destination_path)
        
    # If there is any permission issue
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

    # If there is any permission issue
    except PermissionError:
        print("Permission denied.")

    # For other errors
    except:
        print("Error occurred while copying file.")

print("Pictures copied successfully.")
    

Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occurred while copying file.
Directory already exist.
Error occu

In [20]:
df['picture_filename'] = ['picture/'+df['picture_filename'][i] if x == True else 'None' for i, x in enumerate(rows_begining_with_extra)]
# os arquivos de foto do paciente devem ser armazenados junto com os arquivos CSV, na subpasta “picture”.

In [21]:
# Tratamento 'healthinsurance_pack'
df['healthinsurance_pack'] = df['healthinsurance_pack'].replace({np.nan: 0})
df['healthinsurance_pack'] = df['healthinsurance_pack'].astype(int)
df['healthinsurance_pack'] = [x if x <= 35 else 0 for x in df['healthinsurance_pack'] ]
# print(df['healthinsurance_pack'].values)

In [22]:
df_planos = pd.read_csv('desafio-base2-output/planos.csv',index_col = "plan")
df_planos.head(7).T

plan,1,2,3,4,5,6,7
code,9118087,518103,1377066,3346654,429694,1561209,2031645
name,Unimed,Bradesco,Amil,NotreDame,Porto Seguro,SulAmérica,Allianz
account,9823,6813,5061,4479,4058,9954,2695
billsenddayinternal,1,20,21,25,1,13,12
returndaysexternal,20,21,25,29,29,21,22


In [23]:
# Neste caso, como o 'healthinsurance_pack' inicia com 1 e está ordenada é fácil de encontrar seu nome correspondente
index = df['healthinsurance_pack'].values
# print(index)
df['healthinsurance_pack_name'] = [ df_planos['name'][i] if i != 0 else np.nan for i in index ]
df['healthinsurance_pack_code'] = [ df_planos['code'][i].astype(int).astype(str) if i != 0 else np.nan for i in index ]

# print(df['healthinsurance_pack_name'])

In [24]:
df['healthinsurance_pack'] = 'json::[\\n\\t{\\n\\t\\t"name":"'+df['healthinsurance_pack_name']+'",\\n\\t\\t"code":"'+df['healthinsurance_pack_code']+'"\\n\\t}\\n]'

In [25]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","status","patient_home_phone","patient_mobile_phone","arrival_time","start_time","end_time","description","all_day","cancel_reason","patient_email","event_blocked_scheduling","healthinsurance_name","eventprocedure_pack"]]

In [26]:
df.to_csv('desafio-base2-output/event_scheduling.csv',index=False, encoding='utf-8')